1.- # Importar todas la librerias

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os
import time

2.- # Iniciar Sesion en IG

In [70]:
# Cargar variables desde .env
load_dotenv()

chrome_options = Options()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://www.instagram.com/")

# Add account credentials
user_ = os.getenv('IG_USERNAME')
pass_ = os.getenv('IG_PASSWORD')

# Locate username and password input fields
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Input username y password
username.send_keys(user_)
password.send_keys(pass_)

# Locate 'log in' button to click on it
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Esperar a que se complete la carga de la página principal o manejar ventanas emergentes
try:
    # Detección de pop-up de "Guardar información de inicio de sesión"
    save_info_popup = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Ahora no')]"))
    )
    save_info_popup.click()
    print("Pop-up de guardar información de login fue manejado.")
except Exception as e:
    print(f"No se pudo manejar el pop-up: {e}")

try:
    # Detección de pop-up de notificaciones (con los botones "Activar" y "Ahora no")
    notifications_popup = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))
    )
    notifications_popup.click()
    print("Pop-up de notificaciones fue manejado.")
except Exception as e:
    print(f"No se pudo manejar el pop-up: {e}")

# Access your profile
account = 'https://www.instagram.com/' + user_
driver.get(account) 
# A partir de aquí, deberías estar logueado y puedes continuar con tu scraping


Pop-up de guardar información de login fue manejado.
Pop-up de notificaciones fue manejado.


3.- # Get number of followers and following

In [71]:
# Usando un XPath relativo para encontrar los seguidores y seguidos
followers_xpath = "//section/main//header//ul/li[2]//a/span"
following_xpath = "//section/main//header//ul/li[3]//a/span"

# Extraer el número de seguidores con espera
try:
    num_followers = int(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, followers_xpath))
    ).text.replace(',', ''))
    print('Número de seguidores:', int(num_followers))
except Exception as e:
    print(f"No se pudo obtener el número de seguidores: {e}")

# Extraer el número de seguidos con espera
try:
    num_following = int(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, following_xpath))
    ).text.replace(',', ''))
    print('Número de seguidos:', int(num_following))
except Exception as e:
    print(f"No se pudo obtener el número de seguidos: {e}")


Número de seguidores: 132
Número de seguidos: 366


In [72]:
# Hacer clic en el enlace de seguidores
try:
    driver.find_element(By.XPATH, followers_xpath).click()
    print("Opción 'seguidores' clicada.")
except Exception as e:
    print(f"No se pudo hacer clic en 'seguidores': {e}")


Opción 'seguidores' clicada.


4.- # Extract follower accounts

In [74]:

# Detect pop-up window
pop_up_xpath = '//html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]'
pop_up_window = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pop_up_xpath)))

# Scroll through all following
b=0
while num_followers > b:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', pop_up_window)
    b=len(driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade'))
    print(b)
    time.sleep(1)
    


11
11
11
11
11
11
11
11
22
33
43
54
66
77
89
101
113
113
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
132


# Listar mis seguidores:

In [75]:
# Get followers accounts
get_followers = driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade')

followers = []
for i in get_followers:
    seguidor = i.text
    followers.append(seguidor) 
print(f'Seguidores: {len(followers)}\n\n', followers)

Seguidores: 132

 ['evelymxuxa', 'buinilina', 'escalanteesquivelarquitectos', 'gladibeth.28', 'jonathanbovi', 'elizabeth01397', 'elrincondekev', 'ilse_asesorinmobiliario', 'codeguardian', 'leninhurtado16', 'juanlimacordova', 'mganghela', 'nilso_eiinzi_h', 'dorissayagocordova', 'reflejosspa', 'edithicitarg', 'ttatianamera', 'leydibuenocamizan', 'antoniolimacordova', 'tracey_edmond000778', 'izela.tl', 'linconmacedo9', 'delita.cano', 'anyila.yamile', 'webd360', 'edu.fernandez_30', 'alexbustamantesantacruz', 'nicodemos_salazar_23', 'mirella.zuloeta', 'laura_wendlingj', 'it_samanthaweb', 'nortmedic.uniformes', 'forex_trading_peru', 'devops_stagelatam', 'gig.a.dev', 'joel.siesquen', 'sonqorest', 'cespedesmarthaa', 'charlychavezgue', 'a1nz2802', 'mariaedisitacamisanperez', 'erpsol_mex', 'gonzalezannyely', 'marbelispa2024', '_kathee7', 'cumbialisa', 'adriangmori', 'mita.cordova', 'mjames4719', 'ghost_jea', 'edgard.prieto', 'rociocamizan', 'motosjch', 'yahairatorresllacsa', 'cesar_j18', 'rbuter

Lista de seguidores

 ['evelymxuxa', 'buinilina', 'escalanteesquivelarquitectos', 'gladibeth.28', 'jonathanbovi', 'elizabeth01397', 'elrincondekev', 'ilse_asesorinmobiliario', 'codeguardian', 'leninhurtado16', 'juanlimacordova', 'mganghela', 'nilso_eiinzi_h', 'dorissayagocordova', 'reflejosspa', 'edithicitarg', 'ttatianamera', 'leydibuenocamizan', 'antoniolimacordova', 'tracey_edmond000778', 'izela.tl', 'linconmacedo9', 'delita.cano', 'anyila.yamile', 'webd360', 'edu.fernandez_30', 'alexbustamantesantacruz', 'nicodemos_salazar_23', 'mirella.zuloeta', 'laura_wendlingj', 'it_samanthaweb', 'nortmedic.uniformes', 'forex_trading_peru', 'devops_stagelatam', 'gig.a.dev', 'joel.siesquen', 'sonqorest', 'cespedesmarthaa', 'charlychavezgue', 'a1nz2802', 'mariaedisitacamisanperez', 'erpsol_mex', 'gonzalezannyely', 'marbelispa2024', '_kathee7', 'cumbialisa', 'adriangmori', 'mita.cordova', 'mjames4719', 'ghost_jea', 'edgard.prieto', 'rociocamizan', 'motosjch', 'yahairatorresllacsa', 'cesar_j18', 'rbuter', 'smithdch08', 'huatangarisayago', 'poli_barros__63', 'jheraldyn.sanchezalvarez.1', 'leobar_37', 'chinskb', 'pintadofernandezada', 'tuereselamordemivida12345', 'adapinfer', 'ediza___', 'tech_coder90', 'charly_rj', 'yarihana_hs', 'voccsusat', 'reinamarisolcamizan', 'lisset_tar', 'dayanallatassanchez', 'zuloaga_daniel', 'antopizaarro', 'art_alex28', 'cherilsaavedra', 'juan_garcor23', 'romulosuarez39', 'andre.tineo20', 'esthefania_alfonzo', 'kevincelis7', 'johan_ic01', 'peraltaolano', 'luisquiroz_1011', 'yaniracinthia', 'freddy_bn', 'jhonatantorrescuray', 'yarihanahuatangari', 'laurasanchezalvares', 'cristhiansempertegui', 'leylidi_alarcon', 'mileralvarez0598', 'melrolf', 'coronel_morales', 'joserodasfitt', 'antony_sanchez__', 'pogwan06', 'wilitorrespasapera', 'cecigarm', 'erlita_sc', 'estherbazan_', 'sergiomondragonsilva', 'axel_p1', 'alexish.ortiz', 'cecy_valdera', 'david.alvarez11', 'limacamisan', 'omaraga12', 'victor.0404', 'infomitchell', 'marlon_fonse', 'jeandy91', 'xiomara.piscoya', 'liligalvezcamizan', 'leidi.lima172', 'raftjaz', 'alonso_martinez09', 'lizethcamizanfonseca', 'patryck_p_p', 'littleangee', 'oxalcvasquez', 'caritorold', 'javierrodasn', 'yesseniacubasq19', 'jhonatan_campos_r', 'emilyjhasminalvarado', 'steven_vera_c', 'arumi.naomi', 'franprietolima', 'cesar.ribe', 'edenilsonfloresar']

In [76]:
# Access your Instagram account
driver.get(account)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, following_xpath))).click()
print("Opción 'seguidos' clicada.")

#driver.find_element(By.XPATH, following_xpath).click()


Opción 'seguidos' clicada.


In [77]:
# Detect pop-up window
pop_up_xpath = '//html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]'
pop_up_window = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pop_up_xpath)))

# Scroll through all following
b=0
while num_following > b:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', pop_up_window)
    b=len(driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade'))
    time.sleep(1)


In [78]:
# Get following accounts
get_following = driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade')

following = []
for i in get_following:
    seguido = i.text
    following.append(seguido) 
print(f'Following: {len(following)}\n\n', following)

Following: 366

 ['dani__18_mo', 'goncy.js', 'roxanamolinaoficial', 'cherilsaavedra', 'kevincelis7', 'steven_vera_c', 'littleangee', 'leylidi_alarcon', 'emilyjhasminalvarado', 'anyila.yamile', 'mita.cordova', 'rociocamizan', 'a1nz2802', 'gabyfashion1601', 'ttatianamera', 'alonso_martinez09', 'xiomara.piscoya', 'victor.0404', 'alexbustamantesantacruz', 'yarihana_hs', 'raftjaz', 'caritorold', 'tayanahuatangare', 'adriangmori', 'ysamar_gonzales', 'manuel_llontop2409', 'edenilsonfloresar', 'yesseniacubasq19', 'leobar_37', 'mileralvarez0598', 'edithicitarg', 'izela.tl', 'laurasanchezalvares', 'antopizaarro', 'katherlu19', 'charlychavezgue', 'estherbazan_', 'katherinehuatangari', 'nilso_eiinzi_h', 'elizabeth01397', 'dorissayagocordova', 'marlon_fonse', 'arumi.naomi', 'oxalcvasquez', 'javierrodasn', 'freddy_bn', 'edu.fernandez_30', 'leninhurtado16', 'wilitorrespasapera', 'reflejosspa', 'isabelmanayalle', 'naomy2307', 'cesar.ribe', 'atuc_z', 'mendozagalangabriel', 'limacamisan', 'elidaidrogoga

Lista de seguidos:

 ['dani__18_mo', 'goncy.js', 'roxanamolinaoficial', 'cherilsaavedra', 'kevincelis7', 'steven_vera_c', 'littleangee', 'leylidi_alarcon', 'emilyjhasminalvarado', 'anyila.yamile', 'mita.cordova', 'rociocamizan', 'a1nz2802', 'gabyfashion1601', 'ttatianamera', 'alonso_martinez09', 'xiomara.piscoya', 'victor.0404', 'alexbustamantesantacruz', 'yarihana_hs', 'raftjaz', 'caritorold', 'tayanahuatangare', 'adriangmori', 'ysamar_gonzales', 'manuel_llontop2409', 'edenilsonfloresar', 'yesseniacubasq19', 'leobar_37', 'mileralvarez0598', 'edithicitarg', 'izela.tl', 'laurasanchezalvares', 'antopizaarro', 'katherlu19', 'charlychavezgue', 'estherbazan_', 'katherinehuatangari', 'nilso_eiinzi_h', 'elizabeth01397', 'dorissayagocordova', 'marlon_fonse', 'arumi.naomi', 'oxalcvasquez', 'javierrodasn', 'freddy_bn', 'edu.fernandez_30', 'leninhurtado16', 'wilitorrespasapera', 'reflejosspa', 'isabelmanayalle', 'naomy2307', 'cesar.ribe', 'atuc_z', 'mendozagalangabriel', 'limacamisan', 'elidaidrogogalvez', 'keyssi_fernandez', '_kathee7', 'franprietolima', 'angela.arbaiza', 'deyvi_quispe_quesda', 'sergiomondragonsilva', 'pintadofernandezada', 'coronel_morales', 'mayra_asencio.19', 'taqinitechnology', 'anyek.o', 'luciana.huertas.13', 'ediza___', 'william_sv0.o', 'mganghela', 'david.alvarez11', 'jeandy91', 'erlita_sc', 'limacamizan', 'ghost_jea', 'alexish.ortiz', 'andre.tineo20', 'abril_k04', 'luisquiroz_1011', 'jhan_carlos_od', 'joel.siesquen', 'yahairatorresllacsa', 'antony_sanchez__', 'cumbialisa', 'adapinfer', 'axel_p1', 'heydirte', 'jhonatantorrescuray', 'cespedesmarthaa', 'juan_garcor23', 'yarihanahuatangari', 'maritavaxf', 'jheraldyn.sanchezalvarez.1', 'eme0423', 'yselatm28', 'omaraga12', 'esthefania_alfonzo', 'art_alex28', 'cecy_valdera', 'leidi.lima172', 'pogwan06', 'cesar_j18', 'johan_ic01', 'ldiana_lopez', 'yoanalisbethsanchez', 'joserodasfitt', 'jeseniaaa_gll', 'lucero_urpeque', 'thedriann', 'voccsusat', 'cecigarm', 'olinda_calle', 'yaniracinthia', 'paulina0212j', 'jheiner97', 'mirella.zuloeta', 'romulosuarez39', 'huatangarisayago', 'angiesalazarsc', 'anavela_ruizsc', 'milaa_0709', 'lizethcamizanfonseca', 'custodiopuiconenma', 'lisset_tar', 'chinskb', 'amanda.santacruzg', 'ludapazs', 'tatiherreraracho', 'patryck_p_p', 'zuloaga_daniel', 'cristhiansempertegui', 'lili160991', 'britneysilvamera', 'hernanadr', 'gcorcinthya', 'nicodemos_salazar_23', 'smithdch08', 'arwa.pe', 'majo_esquen11', 'jhonatan_campos_r', 'jbradstyle', 'luiggi.1016', 'solano_nicol', 'tuereselamordemivida12345', 'glomise62', '_key_gp03', 'netfiting', 'marinacastilloaltamirano', 'ivangalvezcastillo', 'dianairrazabalp', 'charly_rj', 'briselly05', 'edgard.prieto', 'lilibeth_camizan_ojeda', 'navarro_granda_luzclarita', 'alesstorresr', 'teno.sg', 'shey.sc10', 'jonathanbovi', 'clau.rafaell', '_arii.annaxx_', 'melrolf', 'yeisongamonal', 'sarahiugazllontop', 'issabela_anahy_diaz', 'lucho07_uwu', 'geraltorres23', 'migueltorresbecerra', 'neysmerliza1414', 'annygarcia1016', 'elsy.galvez.927', 'jeremyvrxd', 'buenojaquelinlima', 'sthefany_tcalderon', 'andleo17', 'reinamarisolcamizan', 'infomitchell', 'nicolazprado', 'jheraldine_celis', 'ricardocampos2001', 'juliana_fer99', 'liz_culquipoma', 'uznay_smvoe_ppednaznachenie', 'yafkevin', 'anghelcarrero', 'rayzapazfernandez', 'ivanchinchay_', 'raibranrv', 'banda.08', 'noemidelgadoventura', 'maccs77', 'ilovekimseokjinie', 'arnold_robra12', 'camizanfonseca', 'perueventos_conciertos', 'karlalealflores', 'jhonjairoaltamiranosalazar', 'jhulianamsc', 'jhonnatan_vasquez', 'manuuelf_12', 'huatangarichinchay', 'robertomontero.23', 'xiomartinezgar', 'eduardosantacruzmera', 'nc_karin', 'xio_larrea', 'angek_2.0', 'julio.lar4', 'lily.cap.54', 'laritaguerrero99', 'adrianamelii', 'franzsc14', 'alexis_deza', 'kev_tuesta', 'yovanamera18', 'magdalenasalvador7165', 'javierjsherrera', 'sairaestelacastillo', 'ernestocallechumacero', 'melissagarcia5808', 'maghe_calle', 'florvasquezh', 'gloriach.17', 'abigailvg.18', 'quetalrichi_', 'josep.fonseca.1426', 'edwin_sc21', 'gust_guz', 'jadelacruzg', 'katerine_yadira12', 'fatima.geraldineaj', 'transportes_chiclayo', 'fantasy.cix', 'luceero.13', 'carlos.chung.20', 'perezprinsroy', 'lupelipcuro', 'velezuki', 'edutullume', 'mondragonnicodemos', 'gladibeth.28', 'adi_terrones', 'peraltaolano', 'liligalvezcamizan', 'kelvin_terrones', 'luzdina14', 'nayelisc09', 'franzo433', 'roimerperez18', 'jhoanrl115', 'ronaldortizboy', 'garciamarx11', 'afrodita_0708', 'meraaida', 'greysig.c', 'estefany_mera01', 'ldmtcw', 'yoxandra.22', 'sunarpoficial', 'yohaterrones', 'mainstudio.pe', 'mirtharojas95', 'lexsebaspe', 'jarly_oficial', 'luzpaucare', 'alejandroslim_', 'mariaedisitacamisan', 'arumialc_', 'kellyancajima26', 'yayocode', 'tamymorely', 'lis_gordita', 'jheimy26', 'sweetsaa20', 'lesli.hz', 'diazyulizacubas', 'eduar_g_sc', 'elviscastillotorres', 'kristhieramos', 'haide_alpe', 'marksalazar97', 'elblogdealaskaa', 'valeryluciayf', 'noe_guerodri', 'piscoyron', 'pizzeriavenecia.pe', 'haji_wolf1', 'cipcdlambayeque', 'andrearivera1101', 'nikki_j_maxwell', 'rjcampos14', 'marc.sosa.11', 'codecreativity.podcast', 'licetsantacruzfernandez', 'neidazandobal', 'fredialexislozanocollantes', 'institutodesoftware', 'vanessa.flores.03', 'car.men4272', 'ecuadoraws', 'liliyoselicastillo', 'liderasoftware', 'cristpamela23', 'republicalinux', 'eddav_', 'hackmorelos', 'condorcoders', 'mirellasaouri', 'cristhianlimabueno11', 'erlandrodasramos', 'licet_santa_cruz', 'reflejossalonspa', 'eduardo.limak50', 'wiliancamizan', 'leidilimacamizan', 'laboratoriobarcix', 'dxperuequiposlibres', 'starlink.oficial', '_lauritacastro_', 'reniec_oficial', 'fantasyoficialpe', 'chiclayohonesto', 'nuclioschool', 'luisdeivispakmas', 'huaweimobilepe', 'wowphoneperu', 'cris.hackermentor', 'daben_tattoo', 'tclmobileperu', 'usatenlinea', 'usatchill.pe', 'mipclista', 'ey_peru', 'esmuydenovios', 'sandreke99', 'arturogoga', 'resdinasinagaa', 'matileont', 'elisa_elias__', 'interbank', 'sunatoficial', 'papafinanciero', 'perusmart', 'tectvoficial', 'nohaysinsuerteoficial', 'hacker.mentor', 'mateoglg', 'pchujoy', 'peruconcerts', 'iaenlinea', 'minculturape', 'python.coder_', 'nosomostv', 'teleticket.oficial', 'python.hub', 'jorgelunalunera', 'midu.dev', 'elgrancheffamosos.tv', 'leomessi', 'laurapausini', 'jsconf.cl', 'elsandreke', 'unalukaglobal', 'bicentenariope']

# Step 5: Identify accounts you follow but don't follow you back

In [81]:
# Limpiar las listas eliminando espacios en blanco antes y después de los nombres
clean_followers = [follower.strip() for follower in followers]
clean_following = [follow.strip() for follow in following]

# Calcular quiénes no te siguen
non_followers = list(set(clean_following) - set(clean_followers))

# Mostrar la cantidad de usuarios que no te siguen y sus perfiles
print(f'Usuarios que no me SIGUEN: {len(non_followers)}\n')

# Formatear la salida con enlaces a sus perfiles
for user in non_followers:
    print(f'{user: <50} | https://instagram.com/{user}')


Usuarios que no me SIGUEN: 264

leomessi                                           | https://instagram.com/leomessi
mariaedisitacamisan                                | https://instagram.com/mariaedisitacamisan
liliyoselicastillo                                 | https://instagram.com/liliyoselicastillo
pizzeriavenecia.pe                                 | https://instagram.com/pizzeriavenecia.pe
tatiherreraracho                                   | https://instagram.com/tatiherreraracho
ludapazs                                           | https://instagram.com/ludapazs
roimerperez18                                      | https://instagram.com/roimerperez18
haide_alpe                                         | https://instagram.com/haide_alpe
sandreke99                                         | https://instagram.com/sandreke99
huatangarichinchay                                 | https://instagram.com/huatangarichinchay
ey_peru                                            | https://instagram